## 埋め込みの使用

このノートブックには、OpenAI APIを通じて`text-embedding-3-small`モデルでテキストを埋め込むために使用できる便利なコードスニペットが含まれています。

In [ ]:
from openai import OpenAI
client = OpenAI()

embedding = client.embeddings.create(
    input="Your text goes here", model="text-embedding-3-small"
).data[0].embedding
len(embedding)


1536

API レート制限をより適切に管理するために、'tenacity' パッケージまたは他の指数バックオフ実装の使用を推奨します。APIを短時間で過度に呼び出すとレート制限が発動する可能性があるためです。以下の関数を使用することで、可能な限り高速に埋め込みを取得できます。

In [ ]:
# Negative example (slow and rate-limited)
from openai import OpenAI
client = OpenAI()

num_embeddings = 10000 # Some large number
for i in range(num_embeddings):
    embedding = client.embeddings.create(
        input="Your text goes here", model="text-embedding-3-small"
    ).data[0].embedding
    print(len(embedding))

In [2]:
# Best practice
from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import OpenAI
client = OpenAI()

# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embedding(text: str, model="text-embedding-3-small") -> list[float]:
    return client.embeddings.create(input=[text], model=model).data[0].embedding

embedding = get_embedding("Your text goes here", model="text-embedding-3-small")
print(len(embedding))

1536
